In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para gráficos
sns.set_style("whitegrid")

# ==============================================================================
# 0. CARGA Y UNIFICACIÓN DE DATOS
# ==============================================================================

# URLs de los datos
url1 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

# Cargar los DataFrames
tienda1 = pd.read_csv(url1)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

# Agregar Store_ID
tienda1['Store_ID'] = 1
tienda2['Store_ID'] = 2
tienda3['Store_ID'] = 3
tienda4['Store_ID'] = 4

# Concatenar todos los DataFrames
df_total = pd.concat([tienda1, tienda2, tienda3, tienda4], ignore_index=True)

# Limpieza de nombres de columnas para evitar KeyErrors (quita espacios, tildes y convierte a snake_case)
df_total.columns = df_total.columns.str.strip().str.replace(' ', '_').str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii')

# Nombres de columnas después de la limpieza
col_precio = 'precio'
col_categoria = 'categoria'
col_evaluacion = 'evaluacion_de_compra'
col_producto = 'producto'
col_costo_envio = 'costo_envio'

print("✅ Datos cargados, unificados y nombres de columnas limpiados.")

# ==============================================================================
# 1. ANÁLISIS: Ingreso Total por Tienda
# ==============================================================================

print("\n--- 1. Ingreso Total por Tienda (Suma de Precio) ---")
ingreso_por_tienda = df_total.groupby('store_id')[col_precio].sum().reset_index()
ingreso_por_tienda.rename(columns={col_precio: 'Ingreso_Total'}, inplace=True)
ingreso_por_tienda['Ingreso_Total_Fmt'] = ingreso_por_tienda['Ingreso_Total'].map('${:,.2f}'.format)
print(ingreso_por_tienda[['store_id', 'Ingreso_Total_Fmt']])

# ==============================================================================
# 2. ANÁLISIS: Cantidad de Productos Vendidos por Categoría
# ==============================================================================

print("\n--- 2. Cantidad de Productos Vendidos por Categoría y Tienda (Top 3) ---")
ventas_por_categoria = df_total.groupby(['store_id', col_categoria]).size().reset_index(name='Cantidad_Ventas')

for store_id in df_total['store_id'].unique():
    df_tienda = ventas_por_categoria[ventas_por_categoria['store_id'] == store_id]
    top_categorias = df_tienda.sort_values(by='Cantidad_Ventas', ascending=False).head(3)
    print(f"\nTienda {store_id} (Top 3):")
    print(top_categorias)

# ==============================================================================
# 3. ANÁLISIS: Calificaciones Promedio de Clientes
# ==============================================================================

print("\n--- 3. Calificación Promedio de Clientes por Tienda ---")
calificacion_promedio = df_total.groupby('store_id')[col_evaluacion].mean().round(2).reset_index()
calificacion_promedio.rename(columns={col_evaluacion: 'Calificacion_Promedio'}, inplace=True)
print(calificacion_promedio)

# ==============================================================================
# 4. ANÁLISIS: Productos Más y Menos Vendidos
# ==============================================================================

print("\n--- 4. Productos Más y Menos Vendidos por Tienda ---")
ventas_por_producto = df_total.groupby(['store_id', col_producto]).size().reset_index(name='Cantidad_Ventas')

for store_id in df_total['store_id'].unique():
    df_tienda = ventas_por_producto[ventas_por_producto['store_id'] == store_id]

    mas_vendido = df_tienda.loc[df_tienda['Cantidad_Ventas'].idxmax()]
    menos_vendido = df_tienda.loc[df_tienda['Cantidad_Ventas'].idxmin()]

    print(f"\n--- Tienda {store_id} ---")
    print(f"🥇 Más Vendido: {mas_vendido[col_producto]} ({mas_vendido['Cantidad_Ventas']} ventas)")
    print(f"📉 Menos Vendido: {menos_vendido[col_producto]} ({menos_vendido['Cantidad_Ventas']} ventas)")

# ==============================================================================
# 5. ANÁLISIS: Costo de Envío Promedio
# ==============================================================================

print("\n--- 5. Costo de Envío Promedio por Tienda ---")
costo_envio_promedio = df_total.groupby('store_id')[col_costo_envio].mean().round(2).reset_index()
costo_envio_promedio.rename(columns={col_costo_envio: 'Costo_Envio_Promedio'}, inplace=True)
costo_envio_promedio['Costo_Envio_Promedio_Fmt'] = costo_envio_promedio['Costo_Envio_Promedio'].map('${:,.2f}'.format)
print(costo_envio_promedio[['store_id', 'Costo_Envio_Promedio_Fmt']])

# ==============================================================================
# 6. VISUALIZACIONES
# ==============================================================================

# Unir métricas para visualizaciones
df_metricas = ingreso_por_tienda.merge(calificacion_promedio, on='store_id')
df_metricas = df_metricas.merge(costo_envio_promedio, on='store_id')

# --- GRÁFICO 1: Ingresos Totales por Tienda (Barras) ---
plt.figure(figsize=(10, 6))
sns.barplot(x='store_id', y='Ingreso_Total', data=df_metricas, palette='viridis')

for index, row in df_metricas.iterrows():
    plt.text(row.name, row['Ingreso_Total'] + 100000,
             f'${row["Ingreso_Total"]:,.0f}',
             color='black', ha="center", fontsize=10)

plt.title('1. Ingreso Total por Tienda', fontsize=16)
plt.xlabel('ID de Tienda', fontsize=12)
plt.ylabel('Ingreso Total ($)', fontsize=12)
plt.ticklabel_format(style='plain', axis='y')
plt.show()

# --- GRÁFICO 2: Distribución de Ventas por Categoría (Barras Apiladas) ---
# Calcular las Top 5 categorías globales
top_5_categorias = ventas_por_categoria.groupby(col_categoria)['Cantidad_Ventas'].sum().nlargest(5).index
df_ventas_top = ventas_por_categoria[ventas_por_categoria[col_categoria].isin(top_5_categorias)]

plt.figure(figsize=(12, 7))
sns.barplot(x='store_id', y='Cantidad_Ventas', hue=col_categoria, data=df_ventas_top, dodge=False, palette='Set2')

plt.title('2. Distribución de Ventas por Categoría (Top 5) por Tienda', fontsize=16)
plt.xlabel('ID de Tienda', fontsize=12)
plt.ylabel('Cantidad de Ventas', fontsize=12)
plt.legend(title='Categoría', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# --- GRÁFICO 3: Calificación vs. Costo de Envío (Doble Eje) ---
fig, ax1 = plt.subplots(figsize=(10, 6))

# Eje 1: Calificación Promedio (Línea)
color = 'tab:blue'
ax1.set_xlabel('ID de Tienda')
ax1.set_ylabel('Calificación Promedio (Estrellas)', color=color)
ax1.plot(df_metricas['store_id'], df_metricas['Calificacion_Promedio'], marker='o', color=color, linestyle='--')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim(df_metricas['Calificacion_Promedio'].min() - 0.05, df_metricas['Calificacion_Promedio'].max() + 0.05)
ax1.set_xticks(df_metricas['store_id'])

# Eje 2: Costo de Envío Promedio (Barras)
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Costo de Envío Promedio ($)', color=color)
ax2.bar(df_metricas['store_id'], df_metricas['Costo_Envio_Promedio'], color=color, alpha=0.3, width=0.4)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('3. Satisfacción del Cliente (Calificación) vs. Costo de Envío', fontsize=16)
fig.tight_layout()
plt.show()

✅ Datos cargados, unificados y nombres de columnas limpiados.

--- 1. Ingreso Total por Tienda (Suma de Precio) ---
   store_id  Ingreso_Total_Fmt
0         1  $1,150,880,400.00
1         2  $1,116,343,500.00
2         3  $1,098,019,600.00
3         4  $1,038,375,700.00

--- 2. Cantidad de Productos Vendidos por Categoría y Tienda (Top 3) ---


KeyError: 'categoria'